In [19]:
import pandas as pd
import numpy as np

In [20]:
df_market_cap = pd.read_pickle("UsableData/MarketCap.pkl")
df_employees = pd.read_pickle("UsableData/Employees.pkl")
df_book_value = pd.read_pickle("UsableData/BookValue.pkl")
df_equal = pd.read_pickle("UsableData/Equal.pkl")
df_random = pd.read_pickle("UsableData/Random.pkl")
df_returns = pd.read_pickle("UsableData/Returns.pkl").T
df_eligible = pd.read_pickle("UsableData/EligibleStocks.pkl")

In [21]:
strategies = {"Market Cap": df_market_cap, "Employees": df_employees, "Book Value": df_book_value, "Equal": df_equal, "Random": df_random} # [DATAFRAME, SMALLER_IS_BETTER] e. g. for market cap bigger values mean better mean bigger weight -> False, reversed for variance -> smaller better -> True

In [22]:
# Only run one specific strategy
# STRATEGY = "Book Value" # Select Strategy
# df = strategies[STRATEGY][0]

# from pandas import DataFrame
# years = df_eligible.columns.to_list()
# years.insert(0, years[0]-1)
# try:
#     df_results = pd.read_pickle("UsableData/Results.pkl")
# except FileNotFoundError:
#     df_results = DataFrame(index=years, columns=strategies.keys())


# cash = 1
# df_results.at[years[0], STRATEGY] = cash
# for year in years[1:]:
#     # Calculate weights
#     eligible_stocks = df_eligible[df_eligible[year] == True][year].index.values
#     weight = df.loc[eligible_stocks, year-1]
#     # print(f"{year}: {np.round(cash, 2)}€")
#     if strategies[STRATEGY][1]: # True if smaller value should have greater weight
#         weight = 1/weight # Reverse
#         weight = weight/weight.sum() # Calculate percentual weight
#     else: # Greater value results in greater weight
#         weight = weight/weight.sum()
#     # Go through month and multiply returns
#     total_cash_gained = 0
#     for ric in eligible_stocks:
#         returns = df_returns[ric][(df_returns.index >= f"{year}-01-01") & (df_returns.index <= f"{year}-12-31")]
#         yearly_return = (np.prod(returns/100 + 1)-1)*100
#         monetary_change = cash*weight[ric]*yearly_return/100
#         # print(f"    {ric} - Weight: {np.round(weight[ric]* 100, 2)} % - Return: {np.round((yearly_change-1)*100, 2)} % - Gain: {np.round(absolute_return, 2)}")
#         total_cash_gained += monetary_change
#     cash += total_cash_gained
#     df_results.at[year, STRATEGY] = cash
# df_results.to_pickle("UsableData/Results.pkl")

In [23]:
# Test Data to validate simulation
# from pandas import DataFrame

# i = ["A", "B", "C"]
# data = {2000: [True, True, True], 2001: [True, True, True], 2002: [True, True, True], 2003: [True, True, True]}
# df_eligible = DataFrame(data=data, index = i)
# times = pd.date_range('2000-01-31', periods=48, freq='M')
# df_returns = DataFrame(data = 1, index=times, columns=i)
# data = {1999: [1, 1, 1], 2000: [1, 1, 1], 2001: [1, 1, 1], 2002: [1, 1, 1]}
# df = DataFrame(data=data, index = i)

In [24]:
# Run strategies
from pandas import DataFrame
years = df_eligible.columns.to_list()
years.insert(0, years[0]-1)
df_results = DataFrame(index=df_returns[(df_returns.index >= f"{years[0]}-12-01") & (df_returns.index <= f"{years[-1]}-12-31")].index, columns=strategies.keys())

for strategy in strategies.keys():
    df = strategies[strategy]
    cash = 1
    df_results.at[f"{years[0]}-12-31", strategy] = cash
    for year in years[1:]:
        cash_up_to_month = 0
        # Calculate weights
        eligible_stocks = df_eligible[df_eligible[year] == True][year].index.values
        weight = df.loc[eligible_stocks, year-1]
        weight = weight/weight.sum()
        # Go through month and multiply returns
        returns = df_returns[(df_returns.index >= f"{year}-01-01") & (df_returns.index <= f"{year}-12-31")]
        for month in returns.index:
            total_cash_gained_up_to_month = 0
            returns_up_to_month = returns[(returns.index >= f"{year}-01-01") & (returns.index <= month)]
            for ric in eligible_stocks:
                stock_return_up_to_month = returns_up_to_month[ric].values
                return_up_to_month = (np.prod(stock_return_up_to_month/100 + 1)-1)*100
                monetary_change = cash*weight[ric]*return_up_to_month/100
                # print(f"    {ric} - Weight: {np.round(weight[ric]* 100, 2)} % - Return: {np.round((yearly_change-1)*100, 2)} % - Gain: {np.round(absolute_return, 2)}")
                total_cash_gained_up_to_month += monetary_change
            cash_up_to_month = total_cash_gained_up_to_month + cash
            df_results.at[month, strategy] = cash_up_to_month
        cash = cash_up_to_month
df_results.to_pickle("UsableData/Results.pkl")